In [1]:
import myModels.GNNAgg as agg
import myModels.GNNArch as arc
import myModels.GNNLayers as layers
import myModels.GNNModel as models
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn.models import MLP
import numpy as np
import torch
import math
import torch.nn.functional as F
from myModels.GNNModel import NodeGNNModels
from myModels.ourModels import NodeIterativeGNN
from myModels.utils import make_uniform_schedule
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sizhuang. Use `wandb login --relogin` to force relogin


True

In [2]:
def add_noise(data, percent=0, seed=None):
    #add random 1's to data
    if percent > 0 and percent <= 1:
        len = np.prod(list(data.x.shape))
        ones = math.floor(len * percent)
        zeros = len - ones
        noise = torch.cat((torch.zeros(zeros), torch.ones(ones)))
        if seed is not None:
            rng = torch.Generator()
            rng.manual_seed(seed)
            noise = noise[torch.randperm(noise.size(0), generator=rng)]
        else:
            noise = noise[torch.randperm(noise.size(0))]
        noise = torch.reshape(noise, data.x.shape)
        data.x += noise
    return data

In [3]:
from torch_geometric.utils import add_remaining_self_loops
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
data = dataset[0]
data = add_noise(data, percent=0.5, seed=12345)
data.edge_index = add_remaining_self_loops(data.edge_index)[0]
num_features = dataset.num_features
num_classes = dataset.num_classes

In [4]:
def accuracy(guess, truth):
    correct = guess == truth
    acc = correct.sum().item() / truth.size(dim=0)
    return acc
def train_epoch(model, data, optimizer, scheduler, device):
    model.train()
    data = data.to(device)
    
    output, _ = model(data)
    print(output.size())
    loss = F.cross_entropy(output[data.train_mask], data.y[data.train_mask])
    pred = F.log_softmax(output[data.train_mask], dim=1).argmax(dim=1)
    acc = accuracy(pred, data.y[data.train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    return loss, acc

def validate_epoch(model, data, device):
    model.eval()
    data = data.to(device)

    output, _ = model(data)
    loss = F.cross_entropy(output[data.val_mask], data.y[data.val_mask])
    pred = F.log_softmax(output[data.val_mask], dim=1).argmax(dim=1)
    acc = accuracy(pred, data.y[data.val_mask])
    return loss, acc

def train(model, data, optimizer, scheduler, config, device):
    
    for epoch in range(config.num_epochs):
        loss_train, acc_train = train_epoch(model, data, optimizer, scheduler, device)
        loss_val, acc_val = validate_epoch(model, data, device)
        print("Epoch {}, train loss: {:.4}, train accuracy: {:.4}, validation loss: {:.4}, validation accuracy: {:.4}".format(epoch+1, loss_train, acc_train, loss_val, acc_val))

        
  
def test(model, data, device):
    model.eval()
    data = data.to(device)

    output, _ = model(data)
    loss = F.cross_entropy(output[data.test_mask], data.y[data.test_mask])
    pred = F.log_softmax(output[data.test_mask], dim=1).argmax(dim=1)
    acc = accuracy(pred, data.y[data.test_mask])
    
    return loss, acc

def exp_per_model(model, data, optimizer, scheduler, config, device):
    
    train(model, data, optimizer, scheduler, config, device)
    loss_test, acc_test = test(model, data, device)
    print("Test loss: {:.4}, Test accuracy: {:.4}".format(loss_test, acc_test))


In [5]:
run_config = {
        'num_iter_layers': 9,
        'learning_rate': 0.002,
        'smooth_fac': 0.7,
        'hid_dim':32,
        'weight_decay':0,
        'num_epochs':200,
        'mlp_dropout':0.5,
        'gnn_dropout':0,
        'dataset_name':'Cora',
        'layer_name':'GCNConv',
        'readout_name':'Max',
        'homo_flag':False,
        'batch_size': 32,
        'arc_name':'IterGNN',
        'encoder_layer_num':1,
        'decoder_layer_num':1,
        'pct_start': 0.2,
        'lr_scheduler':'OneCycleLR',
        'jk': None
    }


In [6]:
wandb.init(job_type="Sweep", 
               project="IterGNN", 
               config=run_config, 
               notes="Sweep for the IterGNN, from McCleary",
               tags=["IterGNN"],
               dir="/vast/palmer/scratch/dijk/sh2748",
               mode='disabled')
config = wandb.config
encoder = MLP(
                in_channels=num_features,
                hidden_channels=config.hid_dim,
                out_channels=config.hid_dim,
                num_layers=config.encoder_layer_num
                )
decoder = MLP(
                    in_channels=config.hid_dim,
                    hidden_channels=config.hid_dim,
                    out_channels=num_classes,
                    num_layers=config.decoder_layer_num
                )
train_schedule = make_uniform_schedule(config.num_iter_layers, config.smooth_fac)
net = NodeIterativeGNN(layer_name=config.layer_name,
                           hidden_size=config.hid_dim,
                           train_schedule=train_schedule,
                           homogeneous_flag=config.homo_flag,
                           mlp_dropout=config.mlp_dropout,
                           gnn_dropout=config.gnn_dropout,
                           readout_name=config.readout_name,
                           encoder=encoder,
                           decoder=decoder,
                           jk=None,
                           use_bn=False,
                           
                           )

In [7]:
data

Data(x=[2708, 1433], edge_index=[2, 13264], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [8]:
output,_ = net(data)
output

tensor([[-0.0660, -0.0265,  0.1537,  ..., -0.0935, -0.1207,  0.2228],
        [-0.1341, -0.1096,  0.1154,  ..., -0.1206, -0.1683,  0.0997],
        [-0.1680,  0.0752,  0.1174,  ..., -0.0057, -0.1583,  0.2424],
        ...,
        [-0.0883, -0.0518,  0.1143,  ..., -0.1366, -0.1562,  0.1995],
        [-0.1709,  0.0064,  0.0676,  ..., -0.1483, -0.0899,  0.1555],
        [-0.1821, -0.0205,  0.0805,  ..., -0.1340, -0.0903,  0.2023]],
       grad_fn=<AddmmBackward0>)

In [15]:
optimizer= torch.optim.Adam(net.parameters(), lr = config.learning_rate, weight_decay=config.weight_decay)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config.learning_rate, total_steps=config.num_epochs)

exp_per_model(net, data, optimizer, scheduler=scheduler, config=config, device='cpu')

torch.Size([2708, 7])
Epoch 1, train loss: 0.1968, train accuracy: 0.9643, validation loss: 1.773, validation accuracy: 0.448
torch.Size([2708, 7])
Epoch 2, train loss: 0.2354, train accuracy: 0.9714, validation loss: 1.836, validation accuracy: 0.43
torch.Size([2708, 7])
Epoch 3, train loss: 0.1919, train accuracy: 0.9643, validation loss: 1.928, validation accuracy: 0.406
torch.Size([2708, 7])
Epoch 4, train loss: 0.1979, train accuracy: 0.9929, validation loss: 2.011, validation accuracy: 0.408
torch.Size([2708, 7])
Epoch 5, train loss: 0.2015, train accuracy: 0.9714, validation loss: 2.05, validation accuracy: 0.396
torch.Size([2708, 7])
Epoch 6, train loss: 0.2447, train accuracy: 0.9286, validation loss: 2.013, validation accuracy: 0.396
torch.Size([2708, 7])
Epoch 7, train loss: 0.1623, train accuracy: 0.9929, validation loss: 1.96, validation accuracy: 0.406
torch.Size([2708, 7])
Epoch 8, train loss: 0.1966, train accuracy: 0.9857, validation loss: 1.931, validation accuracy: 0

In [10]:
net2 = NodeGNNModels(architecture_name=config.arc_name,
                        layer_num=config.num_iter_layers,
                        layer_name=config.layer_name,
                        hidden_size=config.hid_dim,
                        readout_name=config.readout_name,
                        homogeneous_flag=config.homo_flag,
                        confidence_layer_num=1,
                        encoder=encoder,
                        decoder=decoder,
                        jk=None
                        )
optimizer= torch.optim.Adam(net.parameters(), lr = config.learning_rate, weight_decay=config.weight_decay)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config.learning_rate, total_steps=config.num_epochs)

exp_per_model(net2, data, optimizer, scheduler=scheduler, config=config, device='cpu')

torch.Size([2708, 7])
Epoch 1, train loss: 1.97, train accuracy: 0.1571, validation loss: 1.928, validation accuracy: 0.106
torch.Size([2708, 7])
Epoch 2, train loss: 1.962, train accuracy: 0.1571, validation loss: 1.929, validation accuracy: 0.112
torch.Size([2708, 7])
Epoch 3, train loss: 1.955, train accuracy: 0.1571, validation loss: 1.93, validation accuracy: 0.122
torch.Size([2708, 7])
Epoch 4, train loss: 1.949, train accuracy: 0.15, validation loss: 1.932, validation accuracy: 0.13
torch.Size([2708, 7])
Epoch 5, train loss: 1.944, train accuracy: 0.1857, validation loss: 1.935, validation accuracy: 0.162
torch.Size([2708, 7])
Epoch 6, train loss: 1.94, train accuracy: 0.1571, validation loss: 1.939, validation accuracy: 0.166
torch.Size([2708, 7])
Epoch 7, train loss: 1.936, train accuracy: 0.1286, validation loss: 1.943, validation accuracy: 0.166
torch.Size([2708, 7])
Epoch 8, train loss: 1.933, train accuracy: 0.1714, validation loss: 1.947, validation accuracy: 0.102
torch.

In [11]:
output,_ = net2(data)
output

tensor([[-0.2403, -0.6158, -3.4744,  ..., -1.4538, -2.8709, -2.1140],
        [-1.9648, -1.9815, -0.7862,  ...,  6.0566, -1.9283, -0.7978],
        [-1.4293, -2.6786, -1.2874,  ...,  6.0299, -1.8966,  0.3656],
        ...,
        [ 1.1256, -0.6381, -0.4952,  ..., -2.1361, -0.4135, -0.7126],
        [ 2.1991, -1.0460, -2.0728,  ..., -0.4305, -2.2178, -0.1020],
        [ 0.9048, -0.3757, -1.8950,  ..., -0.8069, -1.5370, -0.4801]],
       grad_fn=<AddmmBackward0>)

In [12]:
a = torch.tensor([[[1], [2]]])
b = torch.tensor([[1],[2]])
print(a.size())
print(b.size())
a = a.reshape(b.size())
print(a)

torch.Size([1, 2, 1])
torch.Size([2, 1])
tensor([[1],
        [2]])


In [13]:
from torch_geometric.profile.utils import count_parameters
print(count_parameters(net))
print(count_parameters(net2))

47239
47208
